### NER-augmented RAG

ollama llm 모델 클래스 정의

In [25]:
# ==========================
# Step 0.5: OllamaLLM 정의
# ==========================
import requests
from langchain.llms.base import LLM
from typing import Optional, List, Any

class OllamaLLM(LLM):
    model_name: str = "gpt-oss"
    base_url: str = "http://localhost:11434"
    timeout: int = 300  # 모델 크기 때문에 충분히 넉넉히 설정
    
    def _call(
        self, 
        prompt: str, 
        stop: Optional[List[str]] = None,
        run_manager: Optional[Any] = None
    ) -> str:
        url = f"{self.base_url}/api/generate"
        payload = {
            "model": self.model_name,
            "prompt": prompt,
            "stream": False
        }
        
        try:
            response = requests.post(url, json=payload, timeout=self.timeout)
            response.raise_for_status()
            return response.json()['response']
        except requests.Timeout:
            raise RuntimeError(f"Ollama timed out after {self.timeout}s")
        except Exception as e:
            raise RuntimeError(f"Ollama API error: {str(e)}")
    
    @property
    def _llm_type(self) -> str:
        return "ollama"

print("Step 0.5: OllamaLLM 클래스 정의 완료")


Step 0.5: OllamaLLM 클래스 정의 완료


PDF → RAG → BioNER → 후처리 → 최종 drug list 

In [129]:
# ==========================
# Step 1: 라이브러리 로드
# ==========================
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import re

# ==========================
# Step 2: PDF 로드
# ==========================
pdf_path = "/data1/workspace/pdfs/29.pdf"  # PDF 경로
loader = PyMuPDFLoader(pdf_path)
docs = loader.load()

# ==========================
# Step 3: 텍스트 Chunk 분할
# ==========================
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# Step 4: 임베딩 + FAISS 벡터스토어
# ==========================
embeddings = OllamaEmbeddings(model="nomic-embed-text", base_url="http://localhost:11434")
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)
retriever = vectorstore.as_retriever()



In [130]:
# ==========================
# Step 5: BioNER 모델 로드
# ==========================
ner_model_name = "d4data/biomedical-ner-all"
tokenizer = AutoTokenizer.from_pretrained(ner_model_name)
model = AutoModelForTokenClassification.from_pretrained(
    ner_model_name,
    device_map="auto"
)

ner_pipe = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple"
)

# ==========================
# Step 6: NER 추출 + WordPiece 후처리 함수
# ==========================
def extract_drugs(text, ner_pipe, min_score=0.9):
    ner_results = ner_pipe(text)
    # WordPiece 합치기
    drugs = []
    current_word = ""
    for token in ner_results:
        if token['entity_group'] == 'Medication' and token['score'] >= min_score:
            if token['word'].startswith("##"):
                current_word += token['word'][2:]
            else:
                if current_word:
                    drugs.append(current_word)
                current_word = token['word']
    if current_word:
        drugs.append(current_word)
    # 중복 제거
    drugs = list(set(drugs))
    return drugs

# 테스트
test_text = split_documents[0].page_content
drugs_in_chunk = extract_drugs(test_text, ner_pipe)
# print("Step 6: 첫 번째 Chunk에서 추출된 drug names:", drugs_in_chunk)

# ==========================
# Step 6: NER 추출 + WordPiece 후처리 함수
# ==========================
def extract_drugs(text, ner_pipe, min_score=0.9):
    ner_results = ner_pipe(text)
    # WordPiece 합치기
    drugs = []
    current_word = ""
    for token in ner_results:
        if token['entity_group'] == 'Medication' and token['score'] >= min_score:
            if token['word'].startswith("##"):
                current_word += token['word'][2:]
            else:
                if current_word:
                    drugs.append(current_word)
                current_word = token['word']
    if current_word:
        drugs.append(current_word)
    # 중복 제거
    drugs = list(set(drugs))
    return drugs

# 테스트
test_text = split_documents[0].page_content
drugs_in_chunk = extract_drugs(test_text, ner_pipe)
# print("Step 6: 첫 번째 Chunk에서 추출된 drug names:", drugs_in_chunk)


Device set to use cuda:0


In [131]:
# ==========================
# Step 7: 전체 PDF에서 Drug 추출
# ==========================
all_drugs = []
for i, chunk in enumerate(split_documents):
    chunk_drugs = extract_drugs(chunk.page_content, ner_pipe)
    all_drugs.extend(chunk_drugs)
all_drugs = list(set(all_drugs))

# ==========================
# Step 8: RAG + OllamaLLM 연동 (정리)
# ==========================
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# LLM 객체 생성
llm = OllamaLLM(model_name="gpt-oss", temperature=0)
# print("Step 8: OllamaLLM 객체 생성 완료")

# Prompt 정의
prompt_template = """
You are a biomedical text analysis assistant.

Your task is to extract **only the drugs that were actually tested, administered, or were part of the experiments** in the provided document chunks.
Do NOT include drugs that are mentioned only in the background, references, review, or comparison sections.

==== Document Excerpt Start ====
{context}
==== Document Excerpt End ====

Guidelines:
- Prioritize drugs mentioned in the 'Results' or 'Methods' sections.
- Include drugs mentioned in Figures and Tables only if they were part of the experiments.
- Exclude gene names, proteins, pathways, and drugs only cited from literature.
- Merge WordPiece fragments into full drug names.
- Remove duplicates.
- Output **only the most relevant drug as the first candidate**.
- If multiple drugs are clearly part of the experiment, list up to 3, separated by semicolons.
- If unsure, output the drug that is most central to the experimental results.

Answer:
"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context"]
)

# RetrievalQA 체인 생성
chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True,
    verbose=True
)
print("Step 8: RetrievalQA 체인 생성 완료")

# PDF 전체 text 합치기
all_text = " ".join([chunk.page_content for chunk in split_documents])
# print("Step 8: 모든 chunk 합치기 완료, 길이:", len(all_text))

# RAG + LLM 실행
response = chain({"query": all_text})
rag_drugs = response['result']
print("Step 8: RAG + LLM로 추출된 Drug Names:\n", rag_drugs)


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Step 8: RetrievalQA 체인 생성 완료

> Finished chain.
Step 8: RAG + LLM로 추출된 Drug Names:
 neratinib; everolimus; palbociclib
